In [12]:
import warnings
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import argparse
import numpy as np

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.options.mode.chained_assignment = None

from helpers import *
from settings import *

### Script to get player ids

In [5]:
squad_urls = {
  "india":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-squad-1396344/series-squads",
  "afghan":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/afghanistan-squad-1398042/series-squads",
  "aus":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/australia-squad-1396545/series-squads",
  "eng":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/england-squad-1396558/series-squads",
  "nether":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/netherlands-squad-1396753/series-squads",
  "nz":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/new-zealand-squad-1397557/series-squads",
  "sa":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/south-africa-squad-1396372/series-squads",
  "ban":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/bangladesh-squad-1400121/series-squads",
  "sl":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/sri-lanka-squad-1400061/series-squads",
  "pak":"https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/pakistan-squad-1399493/series-squads"
  }
player_id_dict = {}
for url in squad_urls.values():
  page = requests.get(url)
  bs = BeautifulSoup(page.content, 'lxml')
  tabs = bs.find_all("div",{"class":"ds-grid lg:ds-grid-cols-2"})
  for tab in tabs:
    players = tab.find_all('a',href=True)
    for i in range(len(players)):
      if players[i].text.strip()!='':
        player_id_dict[players[i].text.strip()] = players[i].get("href").split("cricketers/")[1].split("-")[-1]

## Retrieve squad and scorecard for match_id

### Squad

In [112]:
match_id = 1384417

In [113]:
download_gsheet_as_csv(squads_spreadsheet_url, sheet_name = f'{match_id}_squad', download_folder="Squads")
squad_df = pd.read_csv(f"Squads/{match_id}_squad.csv")
upload_df_to_gcs(squad_df, f"Squads/{match_id}_squad.csv", bucket_name='wc-2023')

Sheet '1384417_squad' has been downloaded'


### Scorecard

In [114]:
from get_scorecard import get_scorecard
df = get_scorecard(series_id = 1367856, match_id = match_id)
upload_df_to_gcs(
        df, f"Scorecards/{match_id}_scorecard.csv", bucket_name="wc-2023"
    )

In [115]:
df

,index,Name_batting,Team_batting,Player_id,Desc,Runs_batting,Balls_batting,4s_batting,6s_batting,SR,...,base_points_bowling,pace_points_bowling,milestone_points_bowling,impact_points_bowling,bowling_points,Name,Team,fielding_points,bonus_points,total_points
0,0,Abdullah Shafique,1,922941,c Ngidi b Jansen,9,17,1,0,52.94,...,0.0,0.0,0.0,0.0,0.0,Abdullah Shafique,1,0,0,7.0
1,1,Imam ul Haq,1,568276,c Klaasen b Jansen,12,18,2,0,66.66,...,0.0,0.0,0.0,0.0,0.0,Imam ul Haq,1,0,0,12.0
2,2,Babar Azam,1,348144,c †de Kock b Shamsi,50,65,4,1,76.92,...,0.0,0.0,0.0,0.0,0.0,Babar Azam,1,20,0,97.0
3,3,Mohammad Rizwan,1,323389,c †de Kock b Coetzee,31,27,4,1,114.81,...,0.0,0.0,0.0,0.0,0.0,Mohammad Rizwan,1,20,0,80.0
4,4,Iftikhar Ahmed,1,480603,c Klaasen b Shamsi,21,31,1,1,67.74,...,0.0,-2.0,0.0,4.0,2.0,Iftikhar Ahmed,1,0,0,23.0
5,5,Saud Shakeel,1,652687,c †de Kock b Shamsi,52,52,7,0,100.00,...,0.0,0.0,0.0,0.0,0.0,Saud Shakeel,1,10,0,104.0
6,6,Shadab Khan,1,922943,c Maharaj b Coetzee,43,36,3,2,119.44,...,0.0,0.0,0.0,0.0,0.0,Shadab Khan,1,0,0,81.0
7,7,Mohammad Nawaz,1,348148,c Miller b Jansen,24,24,1,2,100.00,...,0.0,-1.0,0.0,7.0,6.0,Mohammad Nawaz,1,0,0,39.0
8,8,Shaheen Shah Afridi,1,1072470,c Maharaj b Shamsi,2,4,0,0,50.00,...,90.0,30.0,25.0,18.0,163.0,Shaheen Shah Afridi,1,0,0,165.0
9,9,Mohammad Wasim,1,1185538,c †de Kock b Ngidi,7,9,0,1,77.77,...,60.0,20.0,10.0,28.0,118.0,Mohammad Wasim,1,10,0,137.0


## Create standings

In [106]:
all_or_nothing_dict = {
    "Trent Boult":"1384397",
    "Glenn Phillips":"1384397",
    "Mitchell Santner":"1384402",
    "Jonny Bairstow":"1384416",
    }

def update_all_or_nothing(scorecards, all_or_nothing_dict):

    for player, match_id in all_or_nothing_dict.items():
        df = scorecards[f"{match_id}_scorecard"]
        gamble_points = df[df["Name_batting"] == player]["total_points"].values[0]
        if gamble_points>= 100:
            new_points = np.ceil(gamble_points*1.5)
        else:
            new_points = np.ceil((gamble_points-100)*1.5)
        df.loc[df["Name_batting"] == player, "total_points"] = new_points
        scorecards[f"{match_id}_scorecard"] = df 

    return scorecards
    

In [116]:
from get_standings import retrieve_team_info, create_score_df, save_outputs
scorecards = retrieve_scorecards()
match_squad_dicts, squad_dict = retrieve_team_info()
scorecards = update_all_or_nothing(scorecards, all_or_nothing_dict)
(score_df, sum_df, cumsum_df, standings_df, agg_points_df,) = create_score_df(
    scorecards, match_squad_dicts, squad_dict, owner_team_dict, player_id_dict
)
save_outputs(score_df, sum_df, cumsum_df, standings_df, agg_points_df)

In [117]:
standings_df

,Owner,Team,Points,Standings
0,Firi,Fiery Dragons,5891.0,1.0
1,Ash,Come on Ash!,5537.0,2.0
2,Jilla,J Kullis,5270.0,3.0
3,Srini,Jeipomda XI,5065.0,4.0
4,Bhar,Galactic Federation,5037.0,5.0
5,Vaithy,Thayir Saadham,4708.0,6.0
6,Shar,Underdogs,4400.0,7.0
7,Saju,The Wolfpack,4229.0,8.0
8,Mabbu,Malayoor Mambattiyaans,3863.0,9.0
9,Siddhu,Thalapathies,2647.0,10.0


In [108]:
standings_df

,Owner,Team,Points,Standings
0,Firi,Fiery Dragons,5546.0,1.0
1,Ash,Come on Ash!,5480.0,2.0
2,Jilla,J Kullis,5107.0,3.0
3,Srini,Jeipomda XI,4938.0,4.0
4,Bhar,Galactic Federation,4849.0,5.0
5,Vaithy,Thayir Saadham,4354.0,6.0
6,Saju,The Wolfpack,3970.0,7.0
7,Shar,Underdogs,3935.0,8.0
8,Mabbu,Malayoor Mambattiyaans,3782.0,9.0
9,Abhi,Kovai Rousers,2555.0,10.0


In [109]:
sum_df

,Owner,Match_1,Match_2,Match_3,Match_4,Match_5,Match_6,Match_7,Match_8,Match_9,...,Match_16,Match_17,Match_18,Match_19,Match_20,Match_21,Match_22,Match_23,Match_24,Match_25
0,Abhi,43.0,0.0,12.0,0.0,149.0,227.0,181.0,0.0,87.0,...,347.0,115.0,23.0,0.0,144.0,215.0,0.0,0.0,142.0,115.0
1,Ash,15.0,113.0,152.0,352.0,212.0,83.0,184.0,200.0,121.0,...,275.0,108.0,755.0,0.0,408.0,266.0,245.0,384.0,453.0,0.0
2,Bhar,0.0,188.0,0.0,211.0,284.0,13.0,0.0,88.0,367.0,...,0.0,98.0,637.0,15.0,215.0,103.0,17.0,296.0,726.0,58.0
3,Firi,18.0,21.0,416.0,345.0,364.0,0.0,60.0,372.0,302.0,...,27.0,555.0,117.0,278.0,207.0,350.0,183.0,647.0,0.0,308.0
4,Jilla,0.0,369.0,6.0,354.0,450.0,107.0,0.0,949.0,36.0,...,0.0,438.0,314.0,168.0,104.0,181.0,279.0,234.0,243.0,85.0
5,Mabbu,154.0,210.0,25.0,181.0,201.0,124.0,396.0,78.0,341.0,...,117.0,222.0,0.0,254.0,178.0,208.0,75.0,0.0,167.0,162.0
6,Saju,253.0,357.0,469.0,28.0,0.0,106.0,378.0,69.0,44.0,...,228.0,0.0,52.0,94.0,-1.0,56.0,398.0,35.0,64.0,115.0
7,Shar,608.0,176.0,64.0,109.0,0.0,93.0,471.0,56.0,21.0,...,126.0,111.0,207.0,100.0,389.0,288.0,52.0,133.0,39.0,68.0
8,Siddhu,146.0,126.0,114.0,125.0,287.0,-131.0,56.0,75.0,41.0,...,227.0,125.0,26.0,269.0,40.0,125.0,138.0,104.0,63.0,14.0
9,Srini,313.0,60.0,111.0,313.0,23.0,571.0,221.0,126.0,0.0,...,318.0,76.0,197.0,244.0,208.0,169.0,0.0,160.0,210.0,31.0


## Transfers

In [111]:
from settings import transfers_spreadsheet_url, owner_team_dict

for owner in sorted(owner_team_dict.keys()):
    download_gsheet_as_csv(transfers_spreadsheet_url, sheet_name = owner, download_folder="Transfers")
    transfer_df = pd.read_csv(f"Transfers/{owner}.csv")
    upload_df_to_gcs(transfer_df, f"Transfers/{owner}.csv", bucket_name='wc-2023')

Sheet 'Abhi' has been downloaded'
Sheet 'Ash' has been downloaded'
Sheet 'Bhar' has been downloaded'
Sheet 'Firi' has been downloaded'
Sheet 'Jilla' has been downloaded'
Sheet 'Mabbu' has been downloaded'
Sheet 'Saju' has been downloaded'
Sheet 'Shar' has been downloaded'
Sheet 'Siddhu' has been downloaded'
Sheet 'Srini' has been downloaded'
Sheet 'Vaithy' has been downloaded'


In [58]:
series_id = 1367856
URL = (
    "https://www.espncricinfo.com/series/"
    + str(series_id)
    + "/scorecard/"
    + str(match_id)
)
page = requests.get(URL)
bs = BeautifulSoup(page.content, "lxml")

table_body = bs.find_all("tbody")

In [63]:
URL = (
    "https://www.espncricinfo.com/series/"
    + str(series_id)
    + "/scorecard/"
    + str(match_id)
)
page = requests.get(URL)
bs = BeautifulSoup(page.content, "lxml")

table_body = bs.find_all("tbody")
bowler_df = pd.DataFrame(
    columns=[
        "Name",
        "Overs",
        "Maidens",
        "Runs",
        "Wickets",
        "Econ",
        "Dots",
        "4s",
        "6s",
        "Wd",
        "Nb",
        "Team",
        "Player_id",
    ]
)
for i, table in enumerate(table_body[1:4:2]):
    if len(table.find_all(True, {"class": "font-weight-bold match-venue"})) == 0:
        rows = table.find_all("tr")
        for row in rows:
            player_id_col = row.find_all("a", href=True)
            if len(player_id_col) > 0 and "cricketers" in player_id_col[0]["href"]:
                player_id = player_id_col[0]["href"].split("-")[-1]
                cols = row.find_all("td")
                cols = [x.text.strip() for x in cols]
                bowler_df = bowler_df.append(
                    pd.Series(
                        [
                            re.sub(r"\W+", " ", cols[0]).strip(),
                            cols[1],
                            cols[2],
                            cols[3],
                            cols[4],
                            cols[5],
                            cols[6],
                            cols[7],
                            cols[8],
                            cols[9],
                            cols[10],
                            (i == 0) + 1,
                            player_id,
                        ],
                        index=bowler_df.columns,
                    ),
                    ignore_index=True,
                )

batsmen_df = pd.DataFrame(
    columns=["Name", "Desc", "Runs", "Balls", "4s", "6s", "SR", "Team", "Player_id"]
)
for i, table in enumerate(table_body[0:4:2]):
    rows = table.find_all("tr")
    for row in rows:
        player_id_col = row.find_all("a", href=True)
        if len(player_id_col) > 0:
            player_id = player_id_col[0]["href"].split("-")[-1]
        cols = row.find_all("td")
        cols = [x.text.strip() for x in cols]
        if cols[0] == "Extras" or cols[0] == "TOTAL":
            continue
        if len(cols) > 1 and cols[1] == "absent hurt":
            batsmen_df = batsmen_df.append(
                pd.Series(
                    [
                        re.sub(r"\W+", " ", cols[0].split("(c)")[0]).strip(),
                        cols[1],
                        0,
                        0,
                        0,
                        0,
                        0,
                        i + 1,
                        player_id,
                    ],
                    index=batsmen_df.columns,
                ),
                ignore_index=True,
            )
        if len(cols[0].split("Did not bat")) > 1:
            cols = row.find_all("td")
            player_id_col = cols[0].find_all("a", href=True)
            player_ids = [player["href"].split("-")[-1] for player in player_id_col]
            dnb_cols = cols[0].find_all("span")
            dnb = [x.text.strip().split("(c)")[0] for x in dnb_cols]
            dnb = [re.sub(r"\W+", " ", x).strip() for x in dnb]
            dnb = [x for x in dnb if not bool(re.search(r"\d", x)) and x != ""]
            dnb = [i for n, i in enumerate(dnb) if i not in dnb[:n]]
            for idx, dnb_batsman in enumerate(dnb):
                batsmen_df = batsmen_df.append(
                    pd.Series(
                        [dnb_batsman, "DNB", 0, 0, 0, 0, 0, i + 1, player_ids[idx]],
                        index=batsmen_df.columns,
                    ),
                    ignore_index=True,
                )

        elif len(cols) > 2:
            batsmen_df = batsmen_df.append(
                pd.Series(
                    [
                        re.sub(r"\W+", " ", cols[0].split("(c)")[0]).strip(),
                        cols[1],
                        cols[2],
                        cols[3],
                        cols[5],
                        cols[6],
                        cols[7],
                        i + 1,
                        player_id,
                    ],
                    index=batsmen_df.columns,
                ),
                ignore_index=True,
            )



In [64]:
bowler_df

,Name,Overs,Maidens,Runs,Wickets,Econ,Dots,4s,6s,Wd,Nb,Team,Player_id
0,Reece Topley,8.5,0,88,3,9.96,23,10,4,4,0,2,461632
1,David Willey,9,1,61,0,6.77,27,6,2,3,1,2,308251
2,Joe Root,6.1,0,48,0,7.78,6,2,2,0,0,2,303669
3,Gus Atkinson,9,0,60,2,6.66,27,6,2,1,0,2,1039481
4,Mark Wood,7,0,76,0,10.85,9,10,2,1,0,2,351588
5,Adil Rashid,10,0,61,2,6.10,20,4,1,0,0,2,244497
6,Lungi Ngidi,5,1,26,2,5.20,23,2,2,3,0,1,542023
7,Marco Jansen,5,0,35,2,7.00,19,4,2,1,0,1,696401
8,Kagiso Rabada,6,1,38,1,6.33,24,6,1,0,1,1,550215
9,Gerald Coetzee,4,0,35,3,8.75,13,5,1,1,0,1,596010


In [68]:
batsmen_df = batsmen_df.drop([22])

In [69]:
batsmen_df

,Name,Desc,Runs,Balls,4s,6s,SR,Team,Player_id
0,Quinton de Kock,c †Buttler b Topley,4,2,1,0,200.00,1,379143
1,Reeza Hendricks,b Rashid,85,75,9,3,113.33,1,269280
2,Rassie van der Dussen,c Bairstow b Rashid,60,61,8,0,98.36,1,337790
3,Aiden Markram,c Bairstow b Topley,42,44,4,0,95.45,1,600498
4,Heinrich Klaasen,b Atkinson,109,67,12,4,162.68,1,436757
5,David Miller,c Stokes b Topley,5,6,1,0,83.33,1,321777
6,Marco Jansen,not out,75,42,3,6,178.57,1,696401
7,Gerald Coetzee,c sub (LS Livingstone) b Atkinson,3,3,0,0,100.00,1,596010
8,Keshav Maharaj,not out,1,1,0,0,100.00,1,267724
9,Kagiso Rabada,DNB,0,0,0,0,0,1,550215


In [70]:
## Point calculation
bowler_df["Wickets"] = bowler_df["Wickets"].astype(int)
bowler_df["Runs"] = bowler_df["Runs"].astype(int)
bowler_df["Dots"] = bowler_df["Dots"].astype(int)
bowler_df["Maidens"] = bowler_df["Maidens"].astype(int)
bowler_df["Balls"] = (
    bowler_df["Overs"]
    .apply(lambda x: x.split("."))
    .apply(lambda x: int(x[0]) * 6 + int(x[1]) if len(x) > 1 else int(x[0]) * 6)
)

bowler_df["base_points"] = 30 * bowler_df["Wickets"]
bowler_df["pace_points"] = bowler_df["Balls"] - bowler_df["Runs"]
bowler_df["pace_points"] = (
    bowler_df["pace_points"].apply(lambda x: x * 2 if x > 0 else x / 2)
).astype(int)
bowler_df["milestone_points"] = bowler_df["Wickets"].replace(
    {1: 0, 2: 10, 3: 25, 4: 45, 5: 70, 6: 100, 7: 100, 8: 100}
)
bowler_df["impact_points"] = (
    0.5 * bowler_df["Dots"] + bowler_df["Maidens"] * 10
).astype(int)
bowler_df["bowling_points"] = (
    bowler_df["base_points"]
    + bowler_df["pace_points"]
    + bowler_df["milestone_points"]
    + bowler_df["impact_points"]
)

batsmen_df["Runs"] = batsmen_df["Runs"].astype(int)
batsmen_df["Balls"] = batsmen_df["Balls"].astype(int)
batsmen_df["4s"] = batsmen_df["4s"].astype(int)
batsmen_df["6s"] = batsmen_df["6s"].astype(int)
batsmen_df["base_points"] = batsmen_df["Runs"]
batsmen_df["pace_points"] = batsmen_df["Runs"] - 0.9 * batsmen_df["Balls"]
batsmen_df["pace_points"] = (
    batsmen_df["pace_points"].apply(lambda x: x * 2 if x > 0 else x / 2)
).astype(int)
batsmen_df["milestone_points"] = (np.floor(batsmen_df["Runs"] / 25)).replace(
    {1.0: 10, 2.0: 25, 3.0: 45, 4.0: 70, 5.0: 70, 6.0: 100, 7.0: 100, 8.0: 100}
)
batsmen_df["impact_points"] = (
    batsmen_df["4s"]
    + 2 * batsmen_df["6s"]
    + (batsmen_df["Runs"] == 0)
    * (batsmen_df["Desc"] != "not out")
    * (batsmen_df["Desc"] != "DNB")
    * (batsmen_df["Desc"] != "absent hurt")
    * (-5)
)
batsmen_df["batting_points"] = (
    batsmen_df["base_points"]
    + batsmen_df["pace_points"]
    + batsmen_df["milestone_points"]
    + batsmen_df["impact_points"]
)

teams_df = (
    pd.concat(
        [
            batsmen_df[["Name", "Team", "Player_id"]],
            bowler_df[["Name", "Team", "Player_id"]],
        ]
    )
    .drop_duplicates()
    .reset_index(drop=True)
)

fielder_df = teams_df.copy()
fielder_df.loc[:, "fielding_points"] = 0
for team in [1, 2]:
    fielders = []
    for wicket in batsmen_df[batsmen_df["Team"] == team]["Desc"]:
        if wicket.find("c & b") == 0:
            fielders.append(wicket.split("c & b")[1].strip())
        elif wicket.find("c") == 0:
            fielders.append(wicket.split("c ")[1].split(" b ")[0].strip())
        if wicket.find("st") == 0:
            fielders.append(wicket.split("st ")[1].split(" b ")[0].strip())
        if wicket.find("run out") == 0:
            fielders.extend(
                [
                    x.strip()
                    for x in wicket.split("run out")[1]
                    .replace("(", "")
                    .replace(")", "")
                    .split("/")
                ]
            )

    fielders = list(
        filter(lambda x: "sub (" not in x and "sub [" not in x, fielders)
    )
    fielders = [re.sub(r"\W+", " ", fielder).strip() for fielder in fielders]
    fielding_team = [1 if team == 2 else 2]
    for fielder in fielders:
        s = fielder_df.loc[fielder_df["Team"] == fielding_team[0]][
            "Name"
        ].str.contains(fielder)
        if len(s[s].index.values) > 0:  ## Entire name matches
            index_val = s[s].index.values[0]
        elif (
            fielder_df["Name"].str.contains(fielder.split()[1]).sum() == 1
        ):  ## Second name matches with exactly one player
            s = fielder_df["Name"].str.contains(fielder.split()[1])
            index_val = s[s].index.values[0]
        elif (
            (fielder_df["Name"].str.contains(fielder.split()[1]))
            & (fielder_df["Name"].str[0] == fielder.split()[0][0])
        ).sum() == 1:  ## Check for second name match and match of first letter of initial with first letter of name
            s = (fielder_df["Name"].str.contains(fielder.split()[1])) & (
                fielder_df["Name"].str[0] == fielder.split()[0][0]
            )
            index_val = s[s].index.values[0]
        elif (
            fielder_df["Name"].str.contains(fielder.split()[0]).sum() == 1
        ):  ## First name matches with exactly one player
            s = fielder_df["Name"].str.contains(fielder.split()[0])
            index_val = s[s].index.values[0]
        else:
            print("MoM not found")
        fielder_df.loc[index_val, "fielding_points"] += 10

### MOM
fielder_df.loc[:, "bonus_points"] = 0
for item in bs.find_all(
    "div", {"class": "ds-flex ds-justify-between ds-items-center"}
):
    if "Player Of The Match" in item.text:
        mom_id = item.find("a")["href"].split("-")[-1]
        fielder_df.loc[fielder_df["Player_id"] == mom_id, "bonus_points"] += 25

### Winning team points
if table_body[4].find_all("tr")[-1].find_all("td")[0].text == "Points":
    match_points = table_body[4].find_all("tr")[-1].find_all("td")[1].text
    team_name_index = {
        bs.find_all(
            "span", {"class": "ds-text-title-xs ds-font-bold ds-capitalize"}
        )[0].text: 1,
        bs.find_all(
            "span", {"class": "ds-text-title-xs ds-font-bold ds-capitalize"}
        )[1].text: 2,
    }
    if match_points.find("2") != -1:
        winner = match_points.split("2")[0].strip()
        winner_index = team_name_index[winner]
        fielder_df.loc[fielder_df["Team"] == winner_index, "bonus_points"] += 5

total_df = (
    teams_df.merge(batsmen_df, how="left", on=["Name", "Player_id", "Team"])
    .merge(
        bowler_df, how="left", on=["Player_id"], suffixes=("_batting", "_bowling")
    )
    .merge(fielder_df, how="left", on=["Player_id"])
    .fillna(0)
)

total_df["total_points"] = (
    total_df["batting_points"]
    + total_df["bowling_points"]
    + total_df["fielding_points"]
    + total_df["bonus_points"]
)

total_df = total_df.reset_index()


In [71]:
total_df

,index,Name_batting,Team_batting,Player_id,Desc,Runs_batting,Balls_batting,4s_batting,6s_batting,SR,...,base_points_bowling,pace_points_bowling,milestone_points_bowling,impact_points_bowling,bowling_points,Name,Team,fielding_points,bonus_points,total_points
0,0,Quinton de Kock,1,379143,c †Buttler b Topley,4,2,1,0,200.00,...,0.0,0.0,0.0,0.0,0.0,Quinton de Kock,1,20,5,34.0
1,1,Reeza Hendricks,1,269280,b Rashid,85,75,9,3,113.33,...,0.0,0.0,0.0,0.0,0.0,Reeza Hendricks,1,10,5,195.0
2,2,Rassie van der Dussen,1,337790,c Bairstow b Rashid,60,61,8,0,98.36,...,0.0,0.0,0.0,0.0,0.0,Rassie van der Dussen,1,10,5,118.0
3,3,Aiden Markram,1,600498,c Bairstow b Topley,42,44,4,0,95.45,...,0.0,0.0,0.0,0.0,0.0,Aiden Markram,1,0,5,65.0
4,4,Heinrich Klaasen,1,436757,b Atkinson,109,67,12,4,162.68,...,0.0,0.0,0.0,0.0,0.0,Heinrich Klaasen,1,0,30,326.0
5,5,David Miller,1,321777,c Stokes b Topley,5,6,1,0,83.33,...,0.0,0.0,0.0,0.0,0.0,David Miller,1,10,5,21.0
6,6,Marco Jansen,1,696401,not out,75,42,3,6,178.57,...,60.0,-2.0,10.0,9.0,77.0,Marco Jansen,1,0,5,291.0
7,7,Gerald Coetzee,1,596010,c sub (LS Livingstone) b Atkinson,3,3,0,0,100.00,...,90.0,-5.0,25.0,6.0,116.0,Gerald Coetzee,1,0,5,124.0
8,8,Keshav Maharaj,1,267724,not out,1,1,0,0,100.00,...,30.0,-7.0,0.0,2.0,25.0,Keshav Maharaj,1,0,5,31.0
9,9,Kagiso Rabada,1,550215,DNB,0,0,0,0,0,...,30.0,-1.0,0.0,22.0,51.0,Kagiso Rabada,1,20,5,76.0


In [72]:
total_df.to_csv(f"./Scorecards/{match_id}_scorecard.csv", header=True, index=False)

In [73]:
upload_df_to_gcs(
        df, f"Scorecards/{match_id}_scorecard.csv", bucket_name="wc-2023"
    )